# Eliminate Blinking Function and Unit Tests
This notebook includes the Python translation of the `Eliminate_Blinking_De_Loc15_MCMC` MATLAB function and corresponding unit tests.

In [ ]:
import numpy as np

def eliminate_blinking(localizations, frames, resolution, A, deviation_probs, traj_num):
    N = len(frames)
    linked_loc = np.zeros(N, dtype=bool)
    loc_is_blink = np.zeros(N, dtype=bool)
    trajectory = np.zeros(N, dtype=int)
    probs = np.zeros(N)

    loc = np.copy(localizations)
    frame = np.copy(frames)

    for i in range(1, int(round(A)) + 1):
        for ii in range(N):
            investigate = 0
            while ii + investigate < N and frame[ii + investigate] - frame[ii] < i:
                investigate += 1
                if ii + investigate >= N:
                    break
            if ii + investigate >= N:
                continue

            while ii + investigate < N and frame[ii + investigate] - frame[ii] == i:
                if not linked_loc[ii] and not loc_is_blink[ii + investigate]:
                    if trajectory[ii] != 0:
                        iit = np.where(trajectory == trajectory[ii])[0]
                    else:
                        iit = np.array([ii])

                    if trajectory[ii + investigate] != 0:
                        iit2 = np.where(trajectory == trajectory[ii + investigate])[0]
                    else:
                        iit2 = np.array([ii + investigate])

                    d1 = loc[iit]
                    d2 = loc[iit2]
                    dists = np.sqrt(((d1[:, None] - d2) ** 2).sum(axis=2))

                    f1 = np.vstack((frame[iit], np.zeros(len(iit)))).T
                    f2 = np.vstack((frame[iit2], np.zeros(len(iit2)))).T
                    framer = np.sqrt(((f1[:, None] - f2) ** 2).sum(axis=2))

                    dd_t = np.floor(dists / resolution).astype(int)
                    dd_t[dd_t >= deviation_probs.shape[1]] = deviation_probs.shape[1] - 1

                    framer_int = np.floor(framer).astype(int)
                    framer_int[framer_int >= deviation_probs.shape[0]] = deviation_probs.shape[0] - 1

                    indices = (framer_int.ravel(), dd_t.ravel())
                    prob_vals = deviation_probs[indices].reshape(framer.shape)

                    if np.max(framer) < A and np.mean(prob_vals) > 0.5:
                        loc_is_blink[ii + investigate] = True
                        probs[ii + investigate] = np.mean(prob_vals)
                        linked_loc[ii] = True

                        if trajectory[ii] == 0:
                            trajectory[ii] = traj_num[ii]

                        if trajectory[ii + investigate] != 0:
                            inds_of_traj = np.where(trajectory == trajectory[ii + investigate])[0]
                        else:
                            inds_of_traj = np.array([ii + investigate])

                        trajectory[inds_of_traj] = trajectory[ii]
                    elif np.mean(prob_vals) > probs[ii + investigate]:
                        probs[ii + investigate] = np.mean(prob_vals)

                investigate += 1
                if ii + investigate >= N:
                    break

    output_loc = np.ones(N)
    output_loc[loc_is_blink] = 0
    return output_loc, probs

In [ ]:
import unittest
import numpy as np
from eliminate_blinking import eliminate_blinking

class TestEliminateBlinking(unittest.TestCase):
    def test_all_non_blinks(self):
        locs = np.random.rand(20, 2) * 100
        frames = np.arange(1, 21)
        resolution = 1
        A = 1
        deviation_probs = np.ones((10, 10)) * 0.4
        traj_num = np.arange(1, 21)
        locs_result, probs = eliminate_blinking(locs, frames, resolution, A, deviation_probs, traj_num)
        self.assertTrue(np.all(locs_result == 1))

    def test_all_blinks(self):
        locs = np.array([[i, i] for i in range(20)])
        frames = np.array([i for i in range(20)])
        resolution = 1
        A = 2
        deviation_probs = np.ones((10, 10)) * 0.9
        traj_num = np.arange(1, 21)
        locs_result, probs = eliminate_blinking(locs, frames, resolution, A, deviation_probs, traj_num)
        self.assertTrue(np.any(locs_result == 0))
        self.assertTrue(np.all(probs >= 0))

    def test_random_blink_mixture(self):
        np.random.seed(0)
        locs = np.random.rand(20, 2) * 10
        frames = np.random.randint(0, 5, 20)
        resolution = 1
        A = 3
        deviation_probs = np.random.rand(10, 10)
        traj_num = np.arange(1, 21)
        locs_result, probs = eliminate_blinking(locs, frames, resolution, A, deviation_probs, traj_num)
        self.assertEqual(len(locs_result), 20)
        self.assertTrue(np.all((locs_result == 1) | (locs_result == 0)))
        self.assertTrue(np.all(probs >= 0))

if __name__ == "__main__":
    unittest.main()